# Übungsaufgaben 6.1: sed & awk


## 1. Aufgabe `gres`
Schreiben Sie ein Skript `gres.bash`, das für einen regulären Ausdruck
RE (im ERE-Format), eine Ersetzung REPL und einem Dateipfad FILE von
der Kommandozeile, diejenigen Zeilen in der Datei FILE ausgibt, auf
die RE passt. Dabei soll jedes Vorkommen von RE in den ausgegebenen
Zeilen durch REPL ersetzt werden.
```bash
$cat file.txt
Fuchs, du
hast die
Gans gestohlen.
./gres.bash '[A-Z]' '00' file.txt
00uchs, du
00ans gestohlen.
$
```

Wegen den übergebenen regulären Ausdrücken auf der Kommandozeile
können Fehler im Skript auftreten.  Welche Fehler können auftreten und
wie kann man diese Fehler beheben?


#### gres.bash:

In [ ]:
%%bash

#!/bin/bash
# sed & awk 2nd Edition, Dale Dougherty & Arnold Robbins, O'Reilly

if [[ $# -lt 3 ]]; then
    echo "Usage $0 PATTERN REPLACEMENT FILE" >&2
    exit 1
fi

pattern=$1
replacement=$2

if [[ -f "$3" ]]; then
    file=$3
else
    echo $3 is not a file. >&2
    exit 1
fi

## LOESUNG A:
#sed -n -e "s/$pattern/$replacement/gp" "$file"

## LOESUNG B (notwendig, wenn "/" in pattern oder replacement):
A="$(echo | tr '\012' '\001')" #Ersatz NEWLINE mit NUL (nicht-druckbares Steuerzeichen als Separator als universelle Lösung)
sed -n -e "s$A$pattern$A$replacement${A}gp" "$file"

#### Usage:

In [16]:
%%bash
./gres.bash

Usage ./gres.bash PATTERN REPLACEMENT FILE


In [25]:
%%bash
./gres.bash e a file.txt

hast dia
Gans gastohlan


In [26]:
%%bash
./gres.bash '[A-Z]' '00' file.txt

00uchs / du
00ans gestohlen


In [27]:
%%bash
./gres.bash '/' '00' file.txt

Fuchs 00 du


### Erläuterung für eigenes Trennerzeichen:

In [17]:
%%bash
#Fehler wenn kein neues Trennerzeichen (sed -n -e "s/$pattern/$replacement/gp" "$file"):
./gres.bash '/' '00' file.txt

sed: 1: "s///00/gp
": bad flag in substitute command: '0'


In [22]:
%%bash
echo -e "1/2" | sed -n -e s///_/gp

sed: 1: "s///_/gp
": bad flag in substitute command: '_'


In [26]:
%%bash
echo -e "1/2" | sed -n -e s!/!_!gp

1_2



### Erläuterungen zum sed-Befehl:

`sed -n -e s/pattern/replacement/gp`

- `s/.../.../g` globale Ersetzung (alle Matches in einer Zeile)
- `-n`-Option zusammen mit `p`-Befehl: druckt nur die Zeilen aus, in denen Ersetzung vorgenommen wurde
  - (Default-Verhalten sed: Befehl wie Substitute auf jeder Zeile anwenden und Ergebnis ausgeben)
- `-e`-Option für Verkettung von sed-Befehlen, hier nicht unbedingt nötig

#### Beispiele:

In [7]:
%%bash
#einfache Ersetzung
echo -e "foo\nfii" | sed s/o/e/

feo
fii


In [8]:
%%bash
#globale Ersetzung
echo -e "foo\nfii" | sed s/o/e/g

fee
fii


In [9]:
%%bash
#mit n-Option: Unterdrückung Ausgabe
echo -e "foo\nfii" | sed -n s/o/e/g

In [10]:
%%bash
#mit zusätzlichem p: print nur der Zeilen mit Ersetzung
echo -e "foo\nfii" | sed -n s/o/e/gp

fee


In [38]:
%%bash
#Beispiel für Verkettung (Beschränkung auf erste Zeile mit 1q):
echo -e "foo\nfoo" | sed -e s/o/e/g -e 1q

fee


## 2. Aufgabe `dirsize`
Schreiben Sie ein Skript `dirsize.bash`, das für die übergebenen
Verzeichnispfade die Größe (in Kilobytes) des Verzeichnisses berechnet
(= Summe der Größe aller Dateien unterhalb des Verzeichnisses). Sie
können davon ausgehen das sich unter den Verzeichnissen nur Dateien
befinden. Beispiel:
```bash
$./dirsize.bash dir1 dir2
dir1: 10K
dir2: 100K
$
```

Wie kann man das Skript so anpassen, dass es auch rekursiv die Größe
von Unterverzeichnissen mit beachtet?


#### dirsize.bash:

In [ ]:
%%bash

#!/bin/bash

for d in "$@"; do
    if [[ ! -d "$d" ]]; then
        echo "not a dir: $d"
        exit 1
    fi
    size=$(ls -l "$d" | awk 'BEGIN{sum=0} /^-/{sum+=$5} END{print int(sum/1000)}') #Ergebnis ohne Rest (Konvertierung float zu int)
#    size=$(ls -l "$d" | awk '/^-/{sum+=$5} END{print int(sum/1000)}') #Variablen müssen nicht initialisiert werden (Default-Wert 0)
    echo "$d: ${size}K"
done


#### Erläuterung:

- Verarbeitung des tabularen Outputs von ls (Zeilen mit File-Informationen) mit awk
- Summieren der Werte des fünften Feldes $5 (Default-Field-Separator awk: Leerzeichen)

In [36]:
%%bash
ls -l dir1

total 8
-rwxr-xr-x@ 1 user  group     0 21 Okt  2021 a.txt
-rwxr-xr-x@ 1 user  group  1563  3 Mai 16:16 b.txt


#### Usage:

In [41]:
%%bash
./dirsize.bash dir1 dir2

dir1: 1K
dir2: 1K


### Alternative Lösung: rekursiv Unterverzeichnisse berücksichtigen

#### dirsize_rec.bash:

In [ ]:
%%bash

#!/bin/bash

for d in "$@"; do
    if [[ ! -d "$d" ]]; then
        echo "not a dir: $d"
        exit 1
    fi
    for f in $d/*; do
        if [[ -f $f ]]; then #für Dateien: Summierung von Dateigrößen in Verzeichnis
            size=$(cat "$f" | wc -c) #vgl. Aufgabe 1, Übungsaufgaben 1 (kbytes.bash)
            sum=$((sum + size))
        elif [[ -d $f ]]; then #für Unterverzeichnisse: rekursiver Aufruf des Skripts ($0=Pfad) mit bisher berechneter Größe $f
            size=$(bash $0 $f | awk '{print $NF}' | tr -d K) #awk: print Größe (letztes Feld), tr löschen von "K"
            sum=$((sum + size * 1000))
        fi
    done
    echo "$d: $((sum/1000))K"
done


#### Usage:

In [39]:
%%bash
./dirsize_rec.bash dir1 dir2

dir1: 1K
dir2: 3K


## 3. Aufgabe `bigrams.awk`

Schreiben Sie ein awk-Skript `bigrams.awk`, das einen Text von `stdin`
ließt, die Bigramme im Eingabetext berechnet und dann die
Bigrammfolgen mit ihren
[λ-geglätteten](https://cis-sp2021.github.io/06_naive_bayes.pdf)
relativen Häufigkeiten ausgibt. Die Variable λ soll dem Skript auf der
Kommandozeile übergeben werden (Standardwert=1).  Beispiel:

```bash
$ cat file.txt | tr '[:punct:]' ' ' | awk -f bigrams.awk 0.1
...
erste bigram 0.3
zweites bigram 0.1
...
$
```

#### bigrams.awk:

In [ ]:
%%bash 

#!/usr/bin/awk -f

BEGIN {
    lambda = 1
    if (ARGC > 0) { #ARGC: Anzahl Übergabeparameter
        lambda = ARGV[1] #ARGV: Vektor Übergabeparameter
        delete ARGV[1] #Löschen Angabe Lambda-Wert aus ARGV
    }
    SUBSEP = " " #Separator für multidimensionale Arrays
    pre = "" #pre = Vorgängertoken
}

!/^$/ { #wenn keine leere Zeile:
    for (i = 1; i <= NF; i++) { #Tokenisierung der Wörter pro Zeile (NF: Anzahl Felder)
        if (pre == "") { #pre = Vorgängertoken
            pre = $i
        } else {
            bigrams[pre, $i]++ #assoziatives Array mit Bigrammen
            pre = $i
        }
    }
}

END { #Berechnung von relativen Häufigkeiten (optional mit Glättung, Default: 1)
    N = 0
    V = 0
    for (b in bigrams) { #Counts Bigramm-Tokens und -Types
        V++ 
        N += bigrams[b] 
    }
    # printf "N=%d V=%d lambda=%f\n", N, V, lambda
    for (b in bigrams) {
        p = (bigrams[b] + lambda) / (N + V)
        printf "%s %f\n", b, p #formatierte Ausgabe (%s: string, %f: float)
    }
}


#### Usage:

In [10]:
%%bash
cat file.txt | tr '[:punct:]' ' ' | awk -f bigrams.awk #0.3

Fuchs du 0,100000
hast die 0,100000
die Gans 0,100000
Gans gestohlen 0,100000
du hast 0,100000
